In [3]:
import os.path
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

from dateutil import parser
import datetime
import webbrowser

# If modifying these SCOPES, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']


def service_google_calendar():
    creds = None
    flow = InstalledAppFlow.from_client_secrets_file(
        '../credentials_oauth.json', SCOPES)
    #creds = flow.run_local_server(port=0)
    creds = flow.run_console()

    service = build('calendar', 'v3', credentials=creds)
    return service



def get_events(start_date, end_date):
    service = service_google_calendar()

    timeMin = start_date.isoformat() + 'Z'
    timeMax = end_date.isoformat() + 'Z'

    print(f'Getting events from {start_date} to {end_date}')
    events_result = service.events().list(
        calendarId='primary', timeMin=timeMin, timeMax=timeMax,
        maxResults=1000, singleEvents=True,
        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('No events found.')
    data = {}
    i = 0
    for event in events:
        i += 1
        start_str = event['start'].get('dateTime', event['start'].get('date'))
        end_str = event['end'].get('dateTime', event['end'].get('date'))
        description = event.get('description', '')
        title = event.get('summary', '')

        start_dt = parser.parse(start_str)
        end_dt = parser.parse(end_str)
        time = end_dt - start_dt
        start_date = start_dt.date()

        #data[i] = {'description': description, 'time' :time }
        data[i] = {'title': title, 'date': start_date ,'description': description, 'time': time}

    return data


/Users/toma.tanaka/Library/Caches/pypoetry/virtualenvs/google-calendar-ctIqo5RZ-py3.9/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
#import datetime

start_date = datetime.datetime(2023, 9, 1)
end_date = datetime.datetime(2023, 9, 30)
data = get_events(start_date, end_date)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=850777348069-qjum0unqirofdj4e9c0dbla79dmoanir.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=ZUy4nwns0UxclQmPKhH3FsWdSasbCS&prompt=consent&access_type=offline


KeyboardInterrupt: Interrupted by user

In [62]:
import google_auth_oauthlib.flow

print(google_auth_oauthlib.flow.__file__)


/Users/toma.tanaka/Library/Caches/pypoetry/virtualenvs/google-calendar-ctIqo5RZ-py3.9/lib/python3.9/site-packages/google_auth_oauthlib/flow.py


In [51]:
def service_google_calendar():
    # The credentials.json file is expected to be in the same directory.
    flow = InstalledAppFlow.from_client_secrets_file('/Users/toma.tanaka/vscode/google-calendar/credentials_oauth.json', SCOPES)
    creds = flow.run_console()

    service = build('calendar', 'v3', credentials=creds)
    return service

def get_events(start_date, end_date):
    service = service_google_calendar()

    timeMin = start_date.isoformat() + 'Z'
    timeMax = end_date.isoformat() + 'Z'

    print(f'Getting events from {start_date} to {end_date}')
    events_result = service.events().list(
        calendarId='primary', timeMin=timeMin, timeMax=timeMax,
        maxResults=10, singleEvents=True,
        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('No events found.')
    data = {}
    i = 0
    for event in events:
        i += 1
        start_str = event['start'].get('dateTime', event['start'].get('date'))
        end_str = event['end'].get('dateTime', event['end'].get('date'))
        description = event.get('description', '')

        start_dt = parser.parse(start_str)
        end_dt = parser.parse(end_str)
        time = end_dt - start_dt

        data[i] = {'description': description, 'time': time}

    return data

In [9]:
import re
import pandas as pd

def data_to_df(data):
    df = pd.DataFrame()
    for i in data:
        description = data[i]['description']
        pattern = re.compile(r'工数:\[(.*?)\]')
        match = pattern.search(description)
        if match:
            code = match.group(1)
            date = data[i]['date']
            time = data[i]['time']
            new_data = pd.DataFrame([[1, 2, 3]], columns=['A', 'B', 'C'])
            df = pd.concat([df, new_data], ignore_index=True)
            print(time)
            print(date)
            print(code)
    return df

In [10]:
data_to_df(data)

0:30:00
2023-09-20
<span>PJ008374-00</span><span>
1:00:00
2023-09-27
PJ008374-00
0:30:00
2023-09-28
PJ008374-00


In [46]:
import pandas as pd
import re
import datetime

def convert_to_time(decimal_time):
    hours = int(decimal_time)
    minutes = int((decimal_time - hours) * 60)
    return f"{hours:02d}:{minutes:02d}"

def data_to_df(data):
    # 初期の空のデータフレームを作成します
    df = pd.DataFrame()

    # データをループで処理します
    for i in data:
        description = data[i]['description']
        pattern = re.compile(r'工数:\[(.*?)\]')
        match = pattern.search(description)

        # マッチするものがある場合、データフレームを更新します
        if match:
            code = match.group(1)
            date = data[i]['date']
            time = data[i]['time'].total_seconds() / 3600  # 時間を時間単位に変換します（オプション）

            # 新しいデータが既存のデータフレームに存在するかチェックします
            if date in df.index and code in df.columns:
                df.at[date, code] += time  # 時間を加算します
            else:
                # データが存在しない場合、新しい行または列を作成します
                df.at[date, code] = time

    # NaN値を0で埋めます
    df.fillna(0, inplace=True)

    for column in df.columns:
        df[column] = df[column].apply(convert_to_time)

    return df


In [47]:
df = data_to_df(data)

In [48]:
df.head()

,PJ00833-10,PJ008374-00,PJ008333-00
2023-09-25,00:30,00:00,00:00
2023-09-27,00:00,01:30,00:30
2023-09-28,00:00,00:30,00:00


In [45]:
for i in df.columns:
    print(i)

PJ00833-10
PJ008374-00
PJ008333-00


In [34]:
import pandas as pd
import re
import datetime

def convert_to_time(decimal_time):
    hours = int(decimal_time)
    minutes = int((decimal_time - hours) * 60)
    return f"{hours:02d}:{minutes:02d}"

def data_to_df(data):
    df = pd.DataFrame()

    for i in data:
        description = data[i]['description']
        pattern = re.compile(r'工数:\[(.*?)\]')
        match = pattern.search(description)

        if match:
            code = match.group(1)
            date = data[i]['date']
            time = data[i]['time']
            
            # 時間を文字列としてフォーマットします
            hours, remainder = divmod(time.seconds, 3600)
            minutes, _ = divmod(remainder, 60)
            time_str = f"{hours:02}:{minutes:02}"

            if date in df.index and code in df.columns:
                # 既存の時間データを抽出し、新しい時間データを加算します
                existing_time_str = df.at[date, code]
                existing_hours, existing_minutes = map(int, existing_time_str.split(":"))
                existing_time = datetime.timedelta(hours=existing_hours, minutes=existing_minutes)
                total_time = existing_time + time
                
                # 合計時間を文字列としてフォーマットします
                hours, remainder = divmod(total_time.seconds, 3600)
                minutes, _ = divmod(remainder, 60)
                df.at[date, code] = f"{hours:02}:{minutes:02}"
            else:
                # 新しいエントリを作成します（型を明示的に指定します）
                df.at[date, code] = time_str

    df.fillna("00:00", inplace=True)

    for column in ['PJ00833-10', 'PJ008374-00', 'PJ008333-00']:
        df[column] = df[column].apply(convert_to_time)
    
    return df



In [39]:
import pandas as pd
import re
import datetime

def data_to_df(data):
    df = pd.DataFrame()

    for i in data:
        description = data[i]['description']
        pattern = re.compile(r'工数:\[(.*?)\]')
        match = pattern.search(description)

        if match:
            code = match.group(1)
            date = data[i]['date']
            time = data[i]['time']
            
            # 時間を文字列としてフォーマットします
            hours, remainder = divmod(time.seconds, 3600)
            minutes, _ = divmod(remainder, 60)
            time_str = f"{hours:02}:{minutes:02}"

            if date in df.index and code in df.columns:
                # 既存の時間データを抽出し、新しい時間データを加算します
                existing_time_str = df.at[date, code]
                existing_hours, existing_minutes = map(int, str(existing_time_str).split(":"))
                existing_time = datetime.timedelta(hours=existing_hours, minutes=existing_minutes)
                total_time = existing_time + time
                
                # 合計時間を文字列としてフォーマットします
                hours, remainder = divmod(total_time.seconds, 3600)
                minutes, _ = divmod(remainder, 60)
                df.at[date, code] = f"{hours:02}:{minutes:02}"
            else:
                # 新しいエントリを作成します（型を明示的に指定します）
                df.at[date, code] = time_str

    df.fillna("00:00", inplace=True)
    
    return df


In [40]:
df = data_to_df(data)

/var/folders/np/kqyc7p1d5h7fy7n79vwt5k84pt_zsh/T/ipykernel_83731/3656188120.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '00:30' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[date, code] = time_str
/var/folders/np/kqyc7p1d5h7fy7n79vwt5k84pt_zsh/T/ipykernel_83731/3656188120.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '01:00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[date, code] = time_str
/var/folders/np/kqyc7p1d5h7fy7n79vwt5k84pt_zsh/T/ipykernel_83731/3656188120.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '00:30' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[date, code] = time_str


ValueError: invalid literal for int() with base 10: 'nan'

In [37]:
df = data_to_df(data)

/var/folders/np/kqyc7p1d5h7fy7n79vwt5k84pt_zsh/T/ipykernel_83731/3656188120.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '00:30' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[date, code] = time_str
/var/folders/np/kqyc7p1d5h7fy7n79vwt5k84pt_zsh/T/ipykernel_83731/3656188120.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '01:00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[date, code] = time_str
/var/folders/np/kqyc7p1d5h7fy7n79vwt5k84pt_zsh/T/ipykernel_83731/3656188120.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '00:30' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[date, code] = time_str


ValueError: invalid literal for int() with base 10: 'nan'

In [33]:
df.head()

,PJ00833-10,PJ008374-00,PJ008333-00
2023-09-25,0.5,0.0,0.0
2023-09-27,0.0,1.5,0.5
2023-09-28,0.0,0.5,0.0
